# Yelp Review Star Rating Prediction via LLM Prompting

This notebook implements three different prompting approaches to classify Yelp reviews into 1-5 star ratings using Large Language Models (LLMs).

## Objectives
1. Compare Zero-Shot, Few-Shot, and Chain-of-Thought prompting approaches
2. Evaluate accuracy, JSON validity, and latency metrics
3. Identify the best performing approach for rating prediction


## 1. Setup & Imports


In [1]:
import os
from dotenv import load_dotenv
load_dotenv('../task2/backend/.env') 
import json
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import google.generativeai as genai
from datetime import datetime

# Set style for plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Load API key from environment
api_key = os.getenv('GEMINI_API_KEY')
if not api_key:
    raise ValueError("Please set GEMINI_API_KEY environment variable")

genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-pro')

print("✓ Setup complete")


✓ Setup complete


## 2. Dataset Loading & Sampling


In [2]:
# Load the Yelp Reviews dataset
# Note: Download from https://www.kaggle.com/datasets/omkarsabnis/yelp-reviews-dataset
# Place the CSV file in the task1/ directory

dataset_path = 'yelp_reviews.csv'  # Update with actual filename

try:
    df = pd.read_csv(dataset_path)
    print(f"✓ Dataset loaded: {len(df)} rows")
    print(f"Columns: {df.columns.tolist()}")
except FileNotFoundError:
    print("⚠ Dataset file not found. Creating sample dataset for demonstration...")
    # Create a sample dataset structure
    sample_data = {
        'text': [
            "Amazing food and great service! Will definitely come back.",
            "Terrible experience. Food was cold and service was slow.",
            "Pretty good overall, but could be better.",
            "Best restaurant in town! Everything was perfect.",
            "Not worth the money. Very disappointed."
        ] * 40,  # 200 samples
        'stars': [5, 1, 3, 5, 2] * 40
    }
    df = pd.DataFrame(sample_data)
    print(f"✓ Sample dataset created: {len(df)} rows")

# Display sample
print("\nSample reviews:")
print(df.head())


⚠ Dataset file not found. Creating sample dataset for demonstration...
✓ Sample dataset created: 200 rows

Sample reviews:
                                                text  stars
0  Amazing food and great service! Will definitel...      5
1  Terrible experience. Food was cold and service...      1
2          Pretty good overall, but could be better.      3
3   Best restaurant in town! Everything was perfect.      5
4            Not worth the money. Very disappointed.      2


In [3]:
# Sample ~200 rows and split into test set
np.random.seed(42)
sample_size = min(200, len(df))

if len(df) > sample_size:
    df_sample = df.sample(n=sample_size, random_state=42).reset_index(drop=True)
else:
    df_sample = df.copy()

# Use all sampled data as test set for consistent evaluation
test_set = df_sample.copy()

print(f"✓ Test set prepared: {len(test_set)} reviews")
print(f"\nRating distribution:")
print(test_set['stars'].value_counts().sort_index())


✓ Test set prepared: 200 reviews

Rating distribution:
stars
1    40
2    40
3    40
5    80
Name: count, dtype: int64


## 3. Helper Functions


In [4]:
def call_llm(prompt: str, max_retries: int = 3) -> str:
    """Call Gemini API with retry logic"""
    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            return response.text
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
                continue
            else:
                print(f"Error after {max_retries} attempts: {e}")
                return None

def parse_json_response(response: str) -> Dict:
    """Parse JSON from LLM response, handling markdown code blocks"""
    if not response:
        return None
    
    # Remove markdown code blocks if present
    response = response.strip()
    if response.startswith('```'):
        # Extract JSON from code block
        lines = response.split('\n')
        json_lines = []
        in_json = False
        for line in lines:
            if line.strip().startswith('```'):
                in_json = not in_json
                continue
            if in_json or (not response.startswith('```json') and not response.startswith('```')):
                json_lines.append(line)
        response = '\n'.join(json_lines)
    
    try:
        return json.loads(response)
    except json.JSONDecodeError:
        # Try to extract JSON object from text
        import re
        json_match = re.search(r'\{[^{}]*"predicted_stars"[^{}]*\}', response, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group())
            except:
                pass
        return None

def evaluate_predictions(actual: List[int], predicted: List[int], latencies: List[float]) -> Dict:
    """Calculate evaluation metrics"""
    # Exact match accuracy
    exact_matches = sum(1 for a, p in zip(actual, predicted) if a == p)
    exact_accuracy = exact_matches / len(actual) if actual else 0
    
    # Within ±1 accuracy
    within_one = sum(1 for a, p in zip(actual, predicted) if abs(a - p) <= 1)
    within_one_accuracy = within_one / len(actual) if actual else 0
    
    # JSON validity rate
    valid_responses = sum(1 for p in predicted if p is not None)
    validity_rate = valid_responses / len(predicted) if predicted else 0
    
    # Latency metrics
    avg_latency = np.mean(latencies) if latencies else 0
    std_latency = np.std(latencies) if latencies else 0
    
    return {
        'exact_accuracy': exact_accuracy,
        'within_one_accuracy': within_one_accuracy,
        'validity_rate': validity_rate,
        'avg_latency': avg_latency,
        'std_latency': std_latency,
        'total_predictions': len(predicted),
        'valid_predictions': valid_responses
    }

print("✓ Helper functions defined")


✓ Helper functions defined


## 4. Approach 1: Zero-Shot Classification

**Rationale**: Direct instruction to classify without examples. Simple and fast, tests the model's inherent understanding of rating scales.


In [5]:
def zero_shot_prompt(review_text: str) -> str:
    """Generate zero-shot classification prompt"""
    prompt = f"""Classify the following Yelp review into a star rating from 1 to 5, where:
- 1 star = Very negative, terrible experience
- 2 stars = Negative, poor experience
- 3 stars = Neutral, average experience
- 4 stars = Positive, good experience
- 5 stars = Very positive, excellent experience

Review: "{review_text}"

Respond with a JSON object in this exact format:
{{
  "predicted_stars": <number between 1 and 5>,
  "explanation": "<brief reasoning for the assigned rating>"
}}"""
    return prompt

print("Zero-shot prompt template:")
print(zero_shot_prompt("Sample review text"))


Zero-shot prompt template:
Classify the following Yelp review into a star rating from 1 to 5, where:
- 1 star = Very negative, terrible experience
- 2 stars = Negative, poor experience
- 3 stars = Neutral, average experience
- 4 stars = Positive, good experience
- 5 stars = Very positive, excellent experience

Review: "Sample review text"

Respond with a JSON object in this exact format:
{
  "predicted_stars": <number between 1 and 5>,
  "explanation": "<brief reasoning for the assigned rating>"
}


In [ ]:
# Run zero-shot classification on test set
print("Running Zero-Shot Classification...")
zero_shot_results = []
zero_shot_latencies = []

for idx, row in test_set.iterrows():
    review_text = str(row['text'])
    actual_rating = int(row['stars'])
    
    prompt = zero_shot_prompt(review_text)
    
    start_time = time.time()
    response = call_llm(prompt)
    latency = time.time() - start_time
    
    zero_shot_latencies.append(latency)
    
    parsed = parse_json_response(response)
    if parsed and 'predicted_stars' in parsed:
        predicted = int(parsed['predicted_stars'])
        explanation = parsed.get('explanation', '')
    else:
        predicted = None
        explanation = ''
    
    zero_shot_results.append({
        'actual': actual_rating,
        'predicted': predicted,
        'explanation': explanation
    })
    
    if (idx + 1) % 20 == 0:
        print(f"  Processed {idx + 1}/{len(test_set)} reviews...")

print("✓ Zero-Shot classification complete")


Running Zero-Shot Classification...
Error after 3 attempts: 404 models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Error after 3 attempts: 404 models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Error after 3 attempts: 404 models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Error after 3 attempts: 404 models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Error after 3 attempts: 404 models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to s

In [ ]:
# Evaluate zero-shot results
zero_shot_predicted = [r['predicted'] if r['predicted'] is not None else 0 for r in zero_shot_results]
zero_shot_actual = [r['actual'] for r in zero_shot_results]

zero_shot_metrics = evaluate_predictions(
    zero_shot_actual,
    zero_shot_predicted,
    zero_shot_latencies
)

print("Zero-Shot Results:")
print(f"  Exact Accuracy: {zero_shot_metrics['exact_accuracy']:.2%}")
print(f"  Within ±1 Accuracy: {zero_shot_metrics['within_one_accuracy']:.2%}")
print(f"  JSON Validity Rate: {zero_shot_metrics['validity_rate']:.2%}")
print(f"  Average Latency: {zero_shot_metrics['avg_latency']:.2f}s")
print(f"  Std Dev Latency: {zero_shot_metrics['std_latency']:.2f}s")


## 5. Approach 2: Few-Shot Classification

**Rationale**: Provide examples for each rating category to guide the model's understanding. Shows the model what good classifications look like through demonstration.


In [ ]:
def few_shot_prompt(review_text: str) -> str:
    """Generate few-shot classification prompt with examples"""
    examples = """
Examples:

Review: "Terrible service, food was cold and tasteless. Never coming back."
Rating: 1 star
Explanation: Very negative experience with poor service and food quality.

Review: "Not great, but not terrible either. Food was okay but service was slow."
Rating: 2 stars
Explanation: Negative experience overall with below-average service.

Review: "It was fine. Nothing special, nothing terrible. Average experience."
Rating: 3 stars
Explanation: Neutral experience, neither positive nor negative.

Review: "Good food and friendly staff. Had a nice time, would recommend."
Rating: 4 stars
Explanation: Positive experience with good food and service.

Review: "Absolutely amazing! Best restaurant experience ever. Perfect food, perfect service, perfect atmosphere!"
Rating: 5 stars
Explanation: Exceptional experience across all aspects.
"""
    
    prompt = f"""Classify the following Yelp review into a star rating from 1 to 5.
{examples}

Now classify this review:
Review: "{review_text}"

Respond with a JSON object in this exact format:
{{
  "predicted_stars": <number between 1 and 5>,
  "explanation": "<brief reasoning for the assigned rating>"
}}"""
    return prompt

print("Few-shot prompt template (showing structure):")
print("Includes 5 examples (one for each rating)")


In [ ]:
# Run few-shot classification on test set
print("Running Few-Shot Classification...")
few_shot_results = []
few_shot_latencies = []

for idx, row in test_set.iterrows():
    review_text = str(row['text'])
    actual_rating = int(row['stars'])
    
    prompt = few_shot_prompt(review_text)
    
    start_time = time.time()
    response = call_llm(prompt)
    latency = time.time() - start_time
    
    few_shot_latencies.append(latency)
    
    parsed = parse_json_response(response)
    if parsed and 'predicted_stars' in parsed:
        predicted = int(parsed['predicted_stars'])
        explanation = parsed.get('explanation', '')
    else:
        predicted = None
        explanation = ''
    
    few_shot_results.append({
        'actual': actual_rating,
        'predicted': predicted,
        'explanation': explanation
    })
    
    if (idx + 1) % 20 == 0:
        print(f"  Processed {idx + 1}/{len(test_set)} reviews...")

print("✓ Few-Shot classification complete")


In [ ]:
# Evaluate few-shot results
few_shot_predicted = [r['predicted'] if r['predicted'] is not None else 0 for r in few_shot_results]
few_shot_actual = [r['actual'] for r in few_shot_results]

few_shot_metrics = evaluate_predictions(
    few_shot_actual,
    few_shot_predicted,
    few_shot_latencies
)

print("Few-Shot Results:")
print(f"  Exact Accuracy: {few_shot_metrics['exact_accuracy']:.2%}")
print(f"  Within ±1 Accuracy: {few_shot_metrics['within_one_accuracy']:.2%}")
print(f"  JSON Validity Rate: {few_shot_metrics['validity_rate']:.2%}")
print(f"  Average Latency: {few_shot_metrics['avg_latency']:.2f}s")
print(f"  Std Dev Latency: {few_shot_metrics['std_latency']:.2f}s")


## 6. Approach 3: Chain-of-Thought (CoT) with Structured Analysis

**Rationale**: Break down the analysis into explicit reasoning steps. This guides the model through a structured thought process before making the final classification, potentially improving accuracy.


In [ ]:
def chain_of_thought_prompt(review_text: str) -> str:
    """Generate chain-of-thought classification prompt"""
    prompt = f"""Analyze the following Yelp review step by step, then classify it into a star rating (1-5).

Review: "{review_text}"

Follow these steps:

1. **Sentiment Analysis**: Determine the overall sentiment (very negative, negative, neutral, positive, very positive)

2. **Aspect Extraction**: Identify key aspects mentioned (food quality, service, ambiance, value, etc.) and note the sentiment for each

3. **Overall Satisfaction Assessment**: Based on the sentiment and aspects, assess the overall customer satisfaction level

4. **Rating Assignment**: Map the satisfaction level to a star rating:
   - Very negative → 1 star
   - Negative → 2 stars
   - Neutral → 3 stars
   - Positive → 4 stars
   - Very positive → 5 stars

Respond with a JSON object in this exact format:
{{
  "predicted_stars": <number between 1 and 5>,
  "explanation": "<brief reasoning that mentions your analysis steps>"
}}"""
    return prompt

print("Chain-of-Thought prompt template:")
print("Includes structured reasoning steps")


In [ ]:
# Run chain-of-thought classification on test set
print("Running Chain-of-Thought Classification...")
cot_results = []
cot_latencies = []

for idx, row in test_set.iterrows():
    review_text = str(row['text'])
    actual_rating = int(row['stars'])
    
    prompt = chain_of_thought_prompt(review_text)
    
    start_time = time.time()
    response = call_llm(prompt)
    latency = time.time() - start_time
    
    cot_latencies.append(latency)
    
    parsed = parse_json_response(response)
    if parsed and 'predicted_stars' in parsed:
        predicted = int(parsed['predicted_stars'])
        explanation = parsed.get('explanation', '')
    else:
        predicted = None
        explanation = ''
    
    cot_results.append({
        'actual': actual_rating,
        'predicted': predicted,
        'explanation': explanation
    })
    
    if (idx + 1) % 20 == 0:
        print(f"  Processed {idx + 1}/{len(test_set)} reviews...")

print("✓ Chain-of-Thought classification complete")


In [ ]:
# Evaluate chain-of-thought results
cot_predicted = [r['predicted'] if r['predicted'] is not None else 0 for r in cot_results]
cot_actual = [r['actual'] for r in cot_results]

cot_metrics = evaluate_predictions(
    cot_actual,
    cot_predicted,
    cot_latencies
)

print("Chain-of-Thought Results:")
print(f"  Exact Accuracy: {cot_metrics['exact_accuracy']:.2%}")
print(f"  Within ±1 Accuracy: {cot_metrics['within_one_accuracy']:.2%}")
print(f"  JSON Validity Rate: {cot_metrics['validity_rate']:.2%}")
print(f"  Average Latency: {cot_metrics['avg_latency']:.2f}s")
print(f"  Std Dev Latency: {cot_metrics['std_latency']:.2f}s")


In [ ]:
# Evaluate chain-of-thought results
cot_predicted = [r['predicted'] if r['predicted'] is not None else 0 for r in cot_results]
cot_actual = [r['actual'] for r in cot_results]

cot_metrics = evaluate_predictions(
    cot_actual,
    cot_predicted,
    cot_latencies
)

print("Chain-of-Thought Results:")
print(f"  Exact Accuracy: {cot_metrics['exact_accuracy']:.2%}")
print(f"  Within ±1 Accuracy: {cot_metrics['within_one_accuracy']:.2%}")
print(f"  JSON Validity Rate: {cot_metrics['validity_rate']:.2%}")
print(f"  Average Latency: {cot_metrics['avg_latency']:.2f}s")
print(f"  Std Dev Latency: {cot_metrics['std_latency']:.2f}s")


## 7. Comparison & Analysis


In [ ]:
# Create comparison table
comparison_data = {
    'Approach': ['Zero-Shot', 'Few-Shot', 'Chain-of-Thought'],
    'Exact Accuracy': [
        zero_shot_metrics['exact_accuracy'],
        few_shot_metrics['exact_accuracy'],
        cot_metrics['exact_accuracy']
    ],
    'Within ±1 Accuracy': [
        zero_shot_metrics['within_one_accuracy'],
        few_shot_metrics['within_one_accuracy'],
        cot_metrics['within_one_accuracy']
    ],
    'JSON Validity Rate': [
        zero_shot_metrics['validity_rate'],
        few_shot_metrics['validity_rate'],
        cot_metrics['validity_rate']
    ],
    'Avg Latency (s)': [
        zero_shot_metrics['avg_latency'],
        few_shot_metrics['avg_latency'],
        cot_metrics['avg_latency']
    ],
    'Std Dev Latency (s)': [
        zero_shot_metrics['std_latency'],
        few_shot_metrics['std_latency'],
        cot_metrics['std_latency']
    ]
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df['Exact Accuracy'] = comparison_df['Exact Accuracy'].apply(lambda x: f"{x:.2%}")
comparison_df['Within ±1 Accuracy'] = comparison_df['Within ±1 Accuracy'].apply(lambda x: f"{x:.2%}")
comparison_df['JSON Validity Rate'] = comparison_df['JSON Validity Rate'].apply(lambda x: f"{x:.2%}")
comparison_df['Avg Latency (s)'] = comparison_df['Avg Latency (s)'].apply(lambda x: f"{x:.2f}")
comparison_df['Std Dev Latency (s)'] = comparison_df['Std Dev Latency (s)'].apply(lambda x: f"{x:.2f}")

print("Comparison Table:")
print(comparison_df.to_string(index=False))


In [ ]:
# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Accuracy Comparison
approaches = ['Zero-Shot', 'Few-Shot', 'Chain-of-Thought']
exact_acc = [zero_shot_metrics['exact_accuracy'], few_shot_metrics['exact_accuracy'], cot_metrics['exact_accuracy']]
within_one_acc = [zero_shot_metrics['within_one_accuracy'], few_shot_metrics['within_one_accuracy'], cot_metrics['within_one_accuracy']]

x = np.arange(len(approaches))
width = 0.35
axes[0, 0].bar(x - width/2, exact_acc, width, label='Exact Match', alpha=0.8)
axes[0, 0].bar(x + width/2, within_one_acc, width, label='Within ±1', alpha=0.8)
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].set_title('Accuracy Comparison')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(approaches)
axes[0, 0].legend()
axes[0, 0].set_ylim([0, 1])

# 2. Latency Comparison
avg_latencies = [zero_shot_metrics['avg_latency'], few_shot_metrics['avg_latency'], cot_metrics['avg_latency']]
axes[0, 1].bar(approaches, avg_latencies, alpha=0.8, color='orange')
axes[0, 1].set_ylabel('Average Latency (seconds)')
axes[0, 1].set_title('Latency Comparison')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. JSON Validity Rate
validity_rates = [zero_shot_metrics['validity_rate'], few_shot_metrics['validity_rate'], cot_metrics['validity_rate']]
axes[1, 0].bar(approaches, validity_rates, alpha=0.8, color='green')
axes[1, 0].set_ylabel('Validity Rate')
axes[1, 0].set_title('JSON Validity Rate')
axes[1, 0].set_ylim([0, 1])
axes[1, 0].tick_params(axis='x', rotation=45)

# 4. Confusion Matrix for best approach
# Determine best approach
best_idx = np.argmax(exact_acc)
best_name = approaches[best_idx]
if best_idx == 0:
    best_pred = zero_shot_predicted
    best_actual = zero_shot_actual
elif best_idx == 1:
    best_pred = few_shot_predicted
    best_actual = few_shot_actual
else:
    best_pred = cot_predicted
    best_actual = cot_actual

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(best_actual, best_pred, labels=[1, 2, 3, 4, 5])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1, 1], 
            xticklabels=[1, 2, 3, 4, 5], yticklabels=[1, 2, 3, 4, 5])
axes[1, 1].set_xlabel('Predicted')
axes[1, 1].set_ylabel('Actual')
axes[1, 1].set_title(f'Confusion Matrix: {best_name}')

plt.tight_layout()
plt.show()


## 8. Discussion & Conclusions

### Approach Comparison

**Zero-Shot Classification:**
- **Why chosen**: Tests the model's inherent understanding without guidance
- **Strengths**: Fastest approach, minimal prompt overhead
- **Weaknesses**: May lack context for edge cases
- **Improvements made**: Clear rating scale definition in prompt

**Few-Shot Classification:**
- **Why chosen**: Provides concrete examples to guide the model
- **Strengths**: Demonstrates expected output format and reasoning
- **Weaknesses**: Longer prompts, slightly slower
- **Improvements made**: Included diverse examples covering all rating levels

**Chain-of-Thought (CoT) Classification:**
- **Why chosen**: Forces structured reasoning before classification
- **Strengths**: More thorough analysis, potentially better accuracy
- **Weaknesses**: Longest prompts, highest latency
- **Improvements made**: Clear step-by-step reasoning framework

### Key Findings

1. **Best Performing Approach**: [Will be determined after running]
   - Based on exact accuracy, validity rate, and latency trade-offs

2. **JSON Validity**: All approaches should maintain high validity rates with proper parsing

3. **Latency Considerations**: Zero-shot is fastest, but accuracy may justify additional latency in other approaches

4. **Accuracy Patterns**: 
   - Exact match accuracy shows precise classification ability
   - Within ±1 accuracy shows if model understands rating scale even if not exact

### Recommendations

- For production: Choose approach based on accuracy vs latency trade-off
- Consider hybrid approach: Use faster method for simple cases, CoT for ambiguous reviews
- Fine-tune prompts based on domain-specific review patterns
